# Import Library

In [1]:
from google.cloud import bigquery
from google.colab import auth

In [2]:
auth.authenticate_user()
print('Authenticated')

project_id = "jason-366210"
client = bigquery.Client(project=project_id)

Authenticated


# Poin penjabaran

## Problem 1: number of orders growth month to month

Berapa jumlah transaksi yang berstatus `Complete` tiap bulan selama Q1 sampai Q3 di tahun 2022? Insight apa yang bisa kamu berikan?

In [42]:
client.query('''SELECT DISTINCT status
                FROM `bigquery-public-data.thelook_ecommerce.orders`''') \
      .to_dataframe()

,status
0,Shipped
1,Complete
2,Returned
3,Cancelled
4,Processing


In [43]:
# what's the data looks like
client.query('''SELECT *
                FROM `bigquery-public-data.thelook_ecommerce.orders`
                WHERE status='Complete'
                LIMIT 10''') \
      .to_dataframe()

,order_id,user_id,status,gender,created_at,returned_at,shipped_at,delivered_at,num_of_item
0,37,29,Complete,F,2021-07-22 10:28:00+00:00,None,2021-07-25 07:34:00+00:00,2021-07-26 13:33:00+00:00,2
1,44,34,Complete,F,2022-09-05 18:27:00+00:00,None,2022-09-07 01:45:00+00:00,2022-09-08 06:15:00+00:00,1
2,48,37,Complete,F,2022-06-01 01:27:00+00:00,None,2022-06-03 08:33:00+00:00,2022-06-03 09:28:00+00:00,1
3,50,38,Complete,F,2022-06-22 04:57:00+00:00,None,2022-06-22 13:07:00+00:00,2022-06-26 18:09:00+00:00,2
4,55,42,Complete,F,2020-01-19 04:04:00+00:00,None,2020-01-21 19:56:00+00:00,2020-01-25 02:25:00+00:00,1
5,58,43,Complete,F,2022-10-20 09:36:17.756279+00:00,None,2022-10-21 20:30:17.756279+00:00,2022-10-21 23:51:17.756279+00:00,1
6,65,50,Complete,F,2021-05-10 05:38:00+00:00,None,2021-05-10 09:42:00+00:00,2021-05-13 02:28:00+00:00,1
7,94,76,Complete,F,2021-01-13 03:46:00+00:00,None,2021-01-14 19:59:00+00:00,2021-01-16 18:13:00+00:00,1
8,98,79,Complete,F,2021-06-15 16:15:00+00:00,None,2021-06-17 17:42:00+00:00,2021-06-22 16:07:00+00:00,3
9,100,82,Complete,F,2022-10-07 00:47:00+00:00,None,2022-10-08 18:14:00+00:00,2022-10-09 06:59:00+00:00,1


In [44]:
# No order with status Complete have shipped_at NULL OR delivered_at NULL.
client.query('''SELECT *
                FROM `bigquery-public-data.thelook_ecommerce.orders`
                WHERE status='Complete' AND (shipped_at IS NULL OR delivered_at IS NULL)
                LIMIT 10''') \
      .to_dataframe()

,order_id,user_id,status,gender,created_at,returned_at,shipped_at,delivered_at,num_of_item


In [45]:
client.query('''SELECT EXTRACT(year from created_at) as year,
                       EXTRACT(month from created_at) as month,
                       COUNT(order_id) as number_of_sales,
                FROM `bigquery-public-data.thelook_ecommerce.orders`
                WHERE status='Complete' 
                 # BETWEEN equals to >= Q1 2022 AND <= Q3 2022
                 AND created_at BETWEEN "2022-01-01" AND "2022-09-30"
                # EXTRACT is an agg function, so we need to agg it with GROUP BY
                GROUP BY year, month
                ORDER BY year ASC, month ASC''') \
      .to_dataframe()

,year,month,number_of_sales
0,2022,1,1099
1,2022,2,1069
2,2022,3,1294
3,2022,4,1327
4,2022,5,1453
5,2022,6,1599
6,2022,7,1765
7,2022,8,2069
8,2022,9,2314


### The Insights

The insights are:
1. The number of sales growth above 2 digits on March, June to September 2022.
2. The number of sales contracted on February 2022.

In [46]:
client.query('''SELECT year,
                       month,
                       NTILE(3) OVER (ORDER BY year, month) AS quarter,
                       number_of_sales,
                       CONCAT(ROUND(((number_of_sales 
                                      / LAG(number_of_sales, 1) OVER (ORDER BY year, month)) 
                                      - 1) 
                                      * 100
                                    , 2)
                              , '%') as month_to_month,
                FROM (SELECT EXTRACT(year from created_at) as year,
                             EXTRACT(month from created_at) as month,
                             COUNT(order_id) as number_of_sales,
                      FROM `bigquery-public-data.thelook_ecommerce.orders`
                      WHERE status='Complete' and created_at
                       # BETWEEN equals to >= Q1 2022 AND <= Q3 2022
                       BETWEEN "2022-01-01" AND "2022-09-30"
                      # EXTRACT is an agg function, so we need to agg it with GROUP BY
                      GROUP BY year, month)
                ORDER BY year ASC, month ASC''') \
      .to_dataframe()

,year,month,quarter,number_of_sales,month_to_month
0,2022,1,1,1099,None
1,2022,2,1,1069,-2.73%
2,2022,3,1,1294,21.05%
3,2022,4,2,1327,2.55%
4,2022,5,2,1453,9.5%
5,2022,6,2,1599,10.05%
6,2022,7,3,1765,10.38%
7,2022,8,3,2069,17.22%
8,2022,9,3,2314,11.84%


## Problem 2: total sales growth month to month

Berikan informasi total penjualan (dalam USD) tiap bulan selama Q1 sampai Q3 di tahun 2022! (Hanya yang transaksi berstatus `Complete`). Apa informasi yan bisa kamu sampaikan?

In [60]:
# check if order_id in table orders is not unique
client.query('''SELECT order_id
                FROM `bigquery-public-data.thelook_ecommerce.orders`
                GROUP BY order_id HAVING COUNT(*) > 1''') \
      .to_dataframe()

,order_id


In [61]:
# check if order_id, product_id in table order_items is not unique.
# conclusion: column sale_price in table orders is not retail_price * quantity
client.query('''SELECT order_id, product_id, COUNT(*) as num_of_product
                FROM `bigquery-public-data.thelook_ecommerce.order_items`
                GROUP BY order_id, product_id HAVING COUNT(*) > 1''') \
      .to_dataframe()

,order_id,product_id,num_of_product
0,110924,25304,2
1,35096,7290,2
2,13587,23998,2
3,2541,11481,2
4,108919,28205,2
5,24693,21964,2
6,37213,22132,2


In [62]:
client.query('''SELECT *
                FROM `bigquery-public-data.thelook_ecommerce.order_items`
                WHERE order_id=51256''') \
      .to_dataframe()

,id,order_id,user_id,product_id,inventory_item_id,status,created_at,shipped_at,delivered_at,returned_at,sale_price
0,74188,51256,41084,15035,199871,Processing,2021-12-30 00:21:03+00:00,None,None,None,18.780001
1,74189,51256,41084,3603,199873,Processing,2021-12-28 00:03:44+00:00,None,None,None,15.990000


### The Insights

The insights are:
1. The total sales growth above 2 digits on March, June to September 2022.

In [63]:
client.query('''SELECT year,
                       month,
                       NTILE(3) OVER (ORDER BY year, month) as quarter,
                       FORMAT("%'.2f", total_sales) as total_sales,
                       CONCAT(ROUND(((total_sales 
                                      / LAG(total_sales, 1) OVER (ORDER BY year, month)) 
                                      - 1)
                                      * 100
                                    , 2)
                              ,'%') as month_to_month
                FROM (SELECT EXTRACT(year from created_at) as year,
                             EXTRACT(month from created_at) as month,
                             SUM(sale_price) as total_sales
                      FROM `bigquery-public-data.thelook_ecommerce.order_items`
                      WHERE status='Complete'
                       AND created_at BETWEEN "2022-01-01" AND "2022-09-30"
                      GROUP BY year, month)
                ORDER BY year ASC, month ASC''') \
      .to_dataframe()

,year,month,quarter,total_sales,month_to_month
0,2022,1,1,"92,062.96",None
1,2022,2,1,"97,262.37",5.65%
2,2022,3,1,"111,808.83",14.96%
3,2022,4,2,"120,885.37",8.12%
4,2022,5,2,"123,911.50",2.5%
5,2022,6,2,"137,504.41",10.97%
6,2022,7,3,"153,006.01",11.27%
7,2022,8,3,"172,957.70",13.04%
8,2022,9,3,"205,457.36",18.79%


## Problem 3: number of active users growth month too month

Berapa user yang melakukan transaksi berstatus `Complete` di tiap bulan dari Q1 sampai Q3 2022? Apa kesimpulanmu?

### The Insights

The insights are:
1. The number of active users growth above 2 digits on March, August, September 2022.
2. The number of active users contracted on February 2022.

In [83]:
client.query('''SELECT year,
                       month,
                       NTILE(3) OVER (ORDER BY year, month) AS quarter,
                       num_of_user,
                       CONCAT(ROUND((((num_of_user 
                                       / LAG(num_of_user, 1) OVER (ORDER BY year, month)) 
                                       - 1)
                                       * 100)
                                    , 2)
                              , '%') AS month_to_month
                FROM (SELECT EXTRACT(YEAR FROM created_at) AS year,
                             EXTRACT(MONTH FROM created_at) AS month,
                             COUNT(DISTINCT user_id) AS num_of_user
                      FROM `bigquery-public-data.thelook_ecommerce.orders`
                      WHERE status='Complete'
                       AND created_at BETWEEN "2022-01-01" AND "2022-09-30"
                      GROUP BY year, month)
                ORDER BY year ASC, month ASC''') \
      .to_dataframe()

,year,month,quarter,num_of_user,month_to_month
0,2022,1,1,1094,None
1,2022,2,1,1062,-2.93%
2,2022,3,1,1280,20.53%
3,2022,4,2,1320,3.13%
4,2022,5,2,1443,9.32%
5,2022,6,2,1583,9.7%
6,2022,7,3,1739,9.85%
7,2022,8,3,2040,17.31%
8,2022,9,3,2267,11.13%


## Problem 4: category with the most sold quantity month to month

Kategori produk apa saja yang paling banyak dibeli (status transaksi: `Complete`) di tiap bulannya selama Q1 sampai Q3 tahun 2022? Berikan insight!

In [55]:
# see what's the data looks like
client.query('''SELECT *
                FROM `bigquery-public-data.thelook_ecommerce.products`
                LIMIT 5''') \
      .to_dataframe()

,id,cost,category,name,brand,retail_price,department,sku,distribution_center_id
0,27569,92.652563,Swim,2XU Men's Swimmers Compression Long Sleeve Top,2XU,150.410004,Men,B23C5765E165D83AA924FA8F13C05F25,1
1,27445,24.719661,Swim,TYR Sport Men's Square Leg Short Swim Suit,TYR,38.990002,Men,2AB7D3B23574C3DEA2BD278AFD0939AB,1
2,27457,15.897600,Swim,TYR Sport Men's Solid Durafast Jammer Swim Suit,TYR,27.600000,Men,8F831227B0EB6C6D09A0555531365933,1
3,27466,17.850000,Swim,TYR Sport Men's Swim Short/Resistance Short Sw...,TYR,30.000000,Men,67317D6DCC4CB778AEB9219565F5456B,1
4,27481,29.408001,Swim,TYR Alliance Team Splice Jammer,TYR,45.950001,Men,213C888198806EF1A0E2BBF2F4855C6C,1


In [56]:
# first query: get num_of_product of each category per year, month
client.query('''SELECT EXTRACT(YEAR FROM created_at) AS year,
                       EXTRACT(MONTH FROM created_at) AS month,
                       b.category,
                       COUNT(b.category) AS num_of_product
                FROM `bigquery-public-data.thelook_ecommerce.order_items` AS a
                INNER JOIN `bigquery-public-data.thelook_ecommerce.products` AS b
                 ON a.product_id=b.id
                WHERE status='Complete'
                 AND created_at BETWEEN "2022-01-01" AND "2022-09-30"
                GROUP BY year, month, b.category
                ORDER BY year, month, num_of_product DESC''') \
      .to_dataframe()

,year,month,category,num_of_product
0,2022,1,Tops & Tees,133
1,2022,1,Jeans,122
2,2022,1,Shorts,98
3,2022,1,Sweaters,97
4,2022,1,Intimates,97
...,...,...,...,...
228,2022,9,Leggings,50
229,2022,9,Skirts,29
230,2022,9,Jumpsuits & Rompers,16
231,2022,9,Suits,12


In [87]:
# second query: get the rank of each category per year, month
client.query('''SELECT *,
                       DENSE_RANK() OVER (PARTITION BY year, month ORDER BY num_of_product DESC) as rank
                FROM (SELECT EXTRACT(YEAR FROM created_at) AS year,
                             EXTRACT(MONTH FROM created_at) AS month,
                             b.category,
                             COUNT(b.category) AS num_of_product
                      FROM `bigquery-public-data.thelook_ecommerce.order_items` AS a
                      INNER JOIN `bigquery-public-data.thelook_ecommerce.products` AS b
                       ON a.product_id=b.id
                      WHERE status='Complete'
                       AND created_at BETWEEN "2022-01-01" AND "2022-09-30"
                      GROUP BY year, month, b.category)
                ORDER BY year, month, rank''') \
      .to_dataframe()

,year,month,category,num_of_product,rank
0,2022,1,Tops & Tees,133,1
1,2022,1,Jeans,122,2
2,2022,1,Shorts,98,3
3,2022,1,Intimates,97,4
4,2022,1,Sweaters,97,4
...,...,...,...,...,...
228,2022,9,Leggings,50,21
229,2022,9,Skirts,29,22
230,2022,9,Jumpsuits & Rompers,16,23
231,2022,9,Suits,12,24


### The Insights

The insights are:
1. Category Jeans is the category that sold the most in quantity on February, April, May, July 2022.
2. Category Intimates is the category that sold the most in quantity on March, June, August 2022.
3. Category Tops & tees is the category that sold the most in quantity on January, September 2022.


In [58]:
# third query: filter rank 1
client.query('''SELECT year, month, category, num_of_product
                FROM (SELECT *,
                             DENSE_RANK() OVER (PARTITION BY year, month ORDER BY num_of_product DESC) as rank
                      FROM (SELECT EXTRACT(YEAR FROM created_at) AS year,
                                   EXTRACT(MONTH FROM created_at) AS month,
                                   b.category,
                                   COUNT(b.category) AS num_of_product
                            FROM `bigquery-public-data.thelook_ecommerce.order_items` AS a
                            INNER JOIN `bigquery-public-data.thelook_ecommerce.products` AS b
                             ON a.product_id=b.id
                            WHERE status='Complete'
                             AND created_at BETWEEN "2022-01-01" AND "2022-09-30"
                            GROUP BY year, month, b.category)
                     )
                WHERE rank=1
                ORDER BY year, month''') \
      .to_dataframe()

,year,month,category,num_of_product
0,2022,1,Tops & Tees,133
1,2022,2,Jeans,110
2,2022,3,Intimates,163
3,2022,4,Jeans,140
4,2022,5,Jeans,162
5,2022,6,Intimates,173
6,2022,7,Jeans,185
7,2022,8,Intimates,215
8,2022,9,Tops & Tees,254


## Problem 5: category with the most revenue month to month

Kategori produk apa saja yang paling banyak mendapatkan pendapatan (status transaksi: `Complete`) di tiap bulannya selama Q1 sampai Q3 tahun 2022? Berikan insight!

Knowldge requirements:
1. in table order_items, order_id is not unique. This is because, the order_items do not have column quantity. So the database design is **each row for exactly 1 quantity**. Therefore, column product_id is not unique as well.

In [64]:
# This shows that order_id is not unique in table order_items.
# the conclusion: each row represent X in quantity of specific product_id
client.query('''SELECT order_id
                FROM `bigquery-public-data.thelook_ecommerce.order_items`
                GROUP BY order_id HAVING COUNT(*) > 1''') \
      .to_dataframe()

,order_id
0,60681
1,83807
2,9254
3,3339
4,112729
...,...
37287,429
37288,2685
37289,58542
37290,24109


In [65]:
# This shows that each row with specific order_id, product_id is not unique.
# the conclusion: each row represent exactly 1 in quantity of product_id per order_id.
client.query('''SELECT order_id, product_id
                FROM `bigquery-public-data.thelook_ecommerce.order_items`
                GROUP BY order_id, product_id HAVING COUNT(*) > 1''') \
      .to_dataframe()

,order_id,product_id
0,110924,25304
1,35096,7290
2,13587,23998
3,2541,11481
4,108919,28205
5,24693,21964
6,37213,22132


In [68]:
# first query: get the total_sales of each category per year, month
client.query('''SELECT EXTRACT(YEAR FROM created_at) AS year,
                       EXTRACT(MONTH FROM created_at) AS month,
                       b.category,
                       SUM(sale_price) AS total_sales,
                FROM `bigquery-public-data.thelook_ecommerce.order_items` AS a
                INNER JOIN `bigquery-public-data.thelook_ecommerce.products` AS b
                 ON a.product_id=b.id
                WHERE status='Complete'
                 AND created_at BETWEEN "2022-01-01" AND "2022-09-30"
                GROUP BY year, month, b.category
                ORDER BY year, month, total_sales DESC''') \
      .to_dataframe()

,year,month,category,total_sales
0,2022,1,Outerwear & Coats,12424.179995
1,2022,1,Jeans,11327.510033
2,2022,1,Sweaters,6492.389982
3,2022,1,Tops & Tees,6109.820026
4,2022,1,Suits & Sport Coats,5960.149998
...,...,...,...,...
228,2022,9,Suits,1610.239986
229,2022,9,Leggings,1303.009992
230,2022,9,Socks & Hosiery,824.250000
231,2022,9,Jumpsuits & Rompers,597.139998


In [69]:
# second query: get the rank
client.query('''SELECT *,
                       DENSE_RANK() OVER (PARTITION BY year, month ORDER BY total_sales DESC) as rank
                FROM (SELECT EXTRACT(YEAR FROM created_at) AS year,
                             EXTRACT(MONTH FROM created_at) AS month,
                             b.category,
                             SUM(sale_price) AS total_sales,
                      FROM `bigquery-public-data.thelook_ecommerce.order_items` AS a
                      INNER JOIN `bigquery-public-data.thelook_ecommerce.products` AS b
                       ON a.product_id=b.id
                      WHERE status='Complete'
                       AND created_at BETWEEN "2022-01-01" AND "2022-09-30"
                      GROUP BY year, month, b.category)''') \
      .to_dataframe()

,year,month,category,total_sales,rank
0,2022,4,Outerwear & Coats,18180.199965,1
1,2022,4,Jeans,13670.360023,2
2,2022,4,Sweaters,9787.710012,3
3,2022,4,Suits & Sport Coats,8227.009974,4
4,2022,4,Fashion Hoodies & Sweatshirts,7417.310016,5
...,...,...,...,...,...
228,2022,6,Suits,977.140005,22
229,2022,6,Leggings,827.309998,23
230,2022,6,Socks & Hosiery,569.129999,24
231,2022,6,Jumpsuits & Rompers,176.130002,25


### The Insights

The insights are:
1. Category Outerwear & Coats is the most sold category in sales month to month
2. Except, Category Jeans is the most sold category in sales on May, August 2022.

In [71]:
# third query: filter rank 1
client.query('''SELECT year, 
                       month, 
                       category, 
                       FORMAT("%'.2f", total_sales) AS total_sales
                FROM (SELECT *,
                             DENSE_RANK() OVER (PARTITION BY year, month ORDER BY total_sales DESC) as rank
                      FROM (SELECT EXTRACT(YEAR FROM created_at) AS year,
                                   EXTRACT(MONTH FROM created_at) AS month,
                                   b.category,
                                   SUM(sale_price) AS total_sales,
                            FROM `bigquery-public-data.thelook_ecommerce.order_items` AS a
                            INNER JOIN `bigquery-public-data.thelook_ecommerce.products` AS b
                             ON a.product_id=b.id
                            WHERE status='Complete'
                             AND created_at BETWEEN "2022-01-01" AND "2022-09-30"
                            GROUP BY year, month, b.category)
                     )
                WHERE rank=1
                ORDER BY year, month''') \
      .to_dataframe()

,year,month,category,total_sales
0,2022,1,Outerwear & Coats,"12,424.18"
1,2022,2,Outerwear & Coats,"12,364.67"
2,2022,3,Outerwear & Coats,"18,047.42"
3,2022,4,Outerwear & Coats,"18,180.20"
4,2022,5,Jeans,"17,387.62"
5,2022,6,Outerwear & Coats,"18,252.90"
6,2022,7,Outerwear & Coats,"22,464.74"
7,2022,8,Jeans,"19,448.44"
8,2022,9,Outerwear & Coats,"26,976.20"


## Problem 6: correlation of number of orders, total sales vs number of users month to month

Dibandingkan dengan jumlah transaksi dan total penjualan, mana yang paling berkaitan dengan jumlah user yang melakukan transaksi? Apa analisis yang dapat kamu jelaskan? *(Hint: Kamu bisa menggunakan korelasi)*

Catatan: tidak disebutkan data yang diolah adalah Q1 - Q3 2022 maupun dengan status Complete. Oleh karena itu, query dibawah tidak akan menggunakan filter tersebut.

In [72]:
# step 1: aggregate number of orders, total sales and number of users per year, month
client.query('''SELECT EXTRACT(YEAR FROM created_at) AS year,
                       EXTRACT(MONTH FROM created_at) AS month,
                       COUNT(DISTINCT order_id) AS num_of_order,
                       SUM(sale_price) AS total_sales,
                       COUNT(DISTINCT user_id) AS num_of_user
                FROM `bigquery-public-data.thelook_ecommerce.order_items`
                GROUP BY year, month
                ORDER BY year, month''') \
      .to_dataframe()

,year,month,num_of_order,total_sales,num_of_user
0,2019,1,28,1.954870e+03,28
1,2019,2,71,6.163460e+03,71
2,2019,3,140,1.158829e+04,139
3,2019,4,238,2.119557e+04,233
4,2019,5,244,2.151369e+04,241
5,2019,6,340,2.859805e+04,336
6,2019,7,414,3.373096e+04,408
7,2019,8,535,4.585674e+04,530
8,2019,9,533,4.569654e+04,529
9,2019,10,702,5.495835e+04,694


### The Insights

If we calculate the number of orders, total sales, number of users per year, month:
1. The correlation between number of orders and number of users is 0.99493. A correlation of 0.99493 means for every 1 user there is at least an 1x increase of orders compared to previous year, month. (1x is unknown but positive)
2. The correlation between total sales and number of users is 0.993864. A correlation of 0.993864 means for every 1 user there is at least an 1x increase of sales compared to previous year, month. (1x is unknown but positive)
3. In this case, number of orders have higher correlation with number of users.
4. A higher correlation does not mean higher increase. It just means there is better linear relationship / for every increase of 1 user, it is MORE LIKELY to increase by 1x the number of order or sales.

#### Proof of insight No. 4:

Focus on `'num_of_order': [1, 2, 4]`
```
df = pd.DataFrame({'year': [2019, 2019, 2019],
                   'month': [1, 2, 3],
                   'num_of_order': [1, 2, 4],
                   'total_sales': [1000, 2000, 3000],
                   'num_of_user': [1, 2, 3]})
display(df.loc[:,['num_of_order', 'total_sales', 'num_of_user']].corr()['num_of_user'][:-1])
df.plot.scatter(x='num_of_order', y='total_sales')

num_of_order    0.981981
total_sales     1.000000
Name: num_of_user, dtype: float64
```

Let's increase the number_of_order by 1, and see the effect to the correlation.
```
df = pd.DataFrame({'year': [2019, 2019, 2019],
                   'month': [1, 2, 3],
                   'num_of_order': [1, 2, 5],
                   'total_sales': [1000, 2000, 3000],
                   'num_of_user': [1, 2, 3]})
display(df.loc[:,['num_of_order', 'total_sales', 'num_of_user']].corr()['num_of_user'][:-1])
df.plot.scatter(x='num_of_order', y='total_sales')

num_of_order    0.960769
total_sales     1.000000
Name: num_of_user, dtype: float64
```

Now, let's increase the number_of_order by 100
```
df = pd.DataFrame({'year': [2019, 2019, 2019],
                   'month': [1, 2, 3],
                   'num_of_order': [1, 2, 100],
                   'total_sales': [1000, 2000, 3000],
                   'num_of_user': [1, 2, 3]})
display(df.loc[:,['num_of_order', 'total_sales', 'num_of_user']].corr()['num_of_user'][:-1])
df.plot.scatter(x='num_of_order', y='total_sales')

num_of_order    0.870388
total_sales     1.000000
Name: num_of_user, dtype: float64
```

In [73]:
# step 2: find correlation
client.query('''SELECT CORR(num_of_order, num_of_user) AS corr_num_of_order, 
                       CORR(total_sales, num_of_user) AS corr_total_sales
                FROM (SELECT EXTRACT(YEAR FROM created_at) AS year,
                             EXTRACT(MONTH FROM created_at) AS month,
                             COUNT(DISTINCT order_id) AS num_of_order,
                             SUM(sale_price) AS total_sales,
                             COUNT(DISTINCT user_id) AS num_of_user
                      FROM `bigquery-public-data.thelook_ecommerce.order_items`
                      GROUP BY year, month
                      ORDER BY year, month)''') \
      .to_dataframe()

,corr_num_of_order,corr_total_sales
0,0.99493,0.993864


# Problem Statement

## 5W questions

### Who is involved in this goal?

1. The SEO engineer (to bring leads)2. The UI/UX engineer (to increase conversion rate)
3. The Front End engineer (to design the ecommerce according to UI/UX engineer's findings)
4. The Product Owner (to design the business logic)
5. The Back End engineer (to implement the business logic)
6. The Quality Assurance engineer (to make sure the product is working properly before The Look release the app)
7. The DevOps engineer (to scale the server accordingly for more active users)
8. The Photographer (to take photo of the product)
9. The Admin staff (to provide customer service or to close the deal).
10. The Packaging staff (to package the order)
11. The Shipper staff (to ship the order)
12. other role to handle complaint and returned order.

Too many stakeholders. Consequently, the responsibility is spread too thin. No one is to be blamed when the goal is not met.

Therefore, the analyst will choose accordingly:
1. the Photographer
2. the Admin staff
3. the Packaging staff
4. the Shipper staff.

### What do I want to accomplish?

Growth in number of orders, total sales, and number of active users with status Complete.

### Where is this goal to be achieved?

The order will be packaged in the warehouse and shipped to the user address. The goal is achieved when the order status is Complete.

Therefore, the goal is achieved in user address.

### When do I want to achieve this goal?

Per month between Q1 to Q3 2022.

### Why do I want to achieve this goal?

Growth in active users should lead to growth in orders and growth in sales.

## Measurable goals

### How much? How many?

The growth should be positive or above 0%. If below 0%, it is not a growth but a contraction.

### How will I know when it is accomplished?

After the Data Analyst calculate the number by the end of every month.

## Achievable goal

### How can I accomplish this goal?

The Photographer, the Admin staff should live by the motto "underpromise and overdeliver". The Packaging staff should read the receipt before packaging the order. The Shipper staff should deliver the package safelly.

### How realistic is the goal, based on other constraints, such as financial factors?

A growth above 0% in orders, sales and active users is realistic. According to the dataset, The Look does not have column for discount. Therefore, the Data Analyst believe the orders are not met with discount offers.

PS: probably we need to check the column sale_price in table order_items - column cost in table products.

### Do I have the resources and capabilities to achieve the goal? If not, what am I missing?

- [x] staffs
- [x] inventories

The Data Anallyst concluded that The Look ecommerce have the resources and capabilities to achieve the goal.

### Have others done it successfully before?

Tokopedia and Shopee have done it successfully, but these competitors use discount offers irrationally. Consequently, although they are the market leaders, their net profit is in the red.

## Relevant goal

Does this seem worthwhile? Yes.

Is this the right time? Yes.

Does this match our other efforts/needs? Yes.

Am I the right person to reach this goal? Yes. I have the Front End, Back End, Data Analyst technical know how.

Is it applicable in the current socio-economic environment? No.

## Time-bound goal

When? Q1 - Q3 2022.

What can I do six months from now? 1. Hire and train the staffs.
2. Take photo and video of the product.

What can I do six weeks from now?
1. Prepare the training materials for the staffs.
2. Analyze the market to choose the product to sell in Q1 - Q3 2022.

What can I do today?
1. Build the ecommerce (SEO, UI/UX, QA, scale, etc).

## SMART Goals

The Photographer, the Admin staff, the Packaging staff, the Shipper staff will increase orders, sales and active users with status Complete above 0% month to month between Q1 to Q3 2022.

# Poin Analisis

## Problem 1

Berikan kesimpulan dari laporan/informasi yang dibuat berdasarkan problem statement/poin penjabaran menggunakan bahasa awam! (boleh dalam beberapa paragraf)

### Conclusion

The number of active users growth above 2 digits on March, August, September 2022. Suprisingly, the number of orders growth above 2 digits on March, June to September 2022. As expected, the total sales above 2 digits on the same month.

Regarding product categories, the categories Jeans, Intimates, Top & Tees consistently have the most sold quantity month to month in Q1 to Q3 2022. However, the category Outerwear & Coats consitently have the most revenue month to month in Q1 to Q3 2022.

# Poin Pertanyaan

## Problem 1

Apakah problem statement yang kamu definisikan di awal dapat terukur ketercapaiannya? berikan pendapatmu

### The Analysis

The Look ecommerce can achieve the goal because according to the Q1 - Q3 2022 data, The Look does achieve it.

My opinion is that the active users growth correlates with the orders growth and the sales growth.

## Problem 2

Berdasarkan hasil analisis yang sudah kamu lakukan dari 6 penjabaran di atas, jika CEO perusahaanmu ingin menargetkan pendapatan di awal kuartal harus mencapai $250,000, apakah masuk akal.

### The Analysis

According to the month to month revenue data (Poin Pejabaran, Problem 2), Q1 2022 revenue reached $301,134.16. Therefore, the CEO target for the Q1 revenue is achievable.

## Problem 3

CEO kamu menargetkan di kuartal 4 ada investor yang dapat menyuntikan dana ke perusahaanmu, dimana investor akan melihat GMV selama 3 kuartal terakhir serta prospek kedepan minimal di kuartal 4 akan seperti apa. Berikan informasi kepada CEO mu berdasarkan trend transaksi, jumlah user yang bertransaksi, dan GMV 3 kuartal terakhir, apakah perusahaanmu layak atau tidak mendapatkan investor baru!

### The Analysis

Number of orders growth quarter to quarter:
1. Q1 2022: 3462 orders
2. Q2 2022: 4379 orders / 26.49% growth
3. Q3 2022: 6148 orders / 40.4% growth

Number of active users growth quarter to quarter:
1. Q1 2022: 3392 active users
2. Q2 2022: 4270 active users / 25.88% growth
3. Q3 2022: 5905 active users / 38.29% growth

GMV growth quarter to quarter:
1. Q1 2022: $301,134.16

2. Q2 2022: $382,301.28 / 26.95% growth

3. Q3 2022: $531,421.07 / 39.01% growth

In the Data Analyst opinion, The Look ecommerce deserves new investor. Although, the Data Analyst made a note that we should only get new investor if that specific investor will become an irrational competitor, offering lots of discount offers if we reject the investor's proposal. This is because, The Look sales growth is not due to irrational discount offers, in fact, The Look never sold products below the retail price or below the cost / losing money per product.

In [74]:
client.query('''SELECT year,
                       quarter,
                       number_of_sales,
                       CONCAT(ROUND(((number_of_sales 
                                      / LAG(number_of_sales, 1) OVER (ORDER BY year, quarter)) 
                                      - 1) 
                                      * 100
                                    , 2)
                              , '%') AS month_to_month,
                FROM (SELECT EXTRACT(YEAR FROM created_at) AS year,
                             EXTRACT(QUARTER FROM created_at) AS quarter,
                             COUNT(DISTINCT order_id) AS number_of_sales,
                      FROM `bigquery-public-data.thelook_ecommerce.orders`
                      WHERE status='Complete' AND created_at
                       # BETWEEN equals to >= Q1 2022 AND <= Q3 2022
                       BETWEEN "2022-01-01" AND "2022-09-30"
                      # EXTRACT is an agg function, so we need to agg it with GROUP BY
                      GROUP BY year, quarter)
                ORDER BY year ASC, quarter ASC''') \
      .to_dataframe()

,year,quarter,number_of_sales,month_to_month
0,2022,1,3462,None
1,2022,2,4379,26.49%
2,2022,3,6148,40.4%


In [75]:
client.query('''SELECT year,
                       quarter,
                       num_of_user,
                       CONCAT(ROUND(((num_of_user 
                                      / LAG(num_of_user, 1) OVER (ORDER BY year, quarter)) 
                                      - 1) 
                                      * 100
                                    , 2)
                              , '%') AS month_to_month,
                FROM (SELECT EXTRACT(YEAR FROM created_at) AS year,
                             EXTRACT(QUARTER FROM created_at) AS quarter,
                             COUNT(DISTINCT user_id) AS num_of_user,
                      FROM `bigquery-public-data.thelook_ecommerce.orders`
                      WHERE status='Complete' AND created_at
                       # BETWEEN equals to >= Q1 2022 AND <= Q3 2022
                       BETWEEN "2022-01-01" AND "2022-09-30"
                      # EXTRACT is an agg function, so we need to agg it with GROUP BY
                      GROUP BY year, quarter)
                ORDER BY year ASC, quarter ASC''') \
      .to_dataframe()

,year,quarter,num_of_user,month_to_month
0,2022,1,3392,None
1,2022,2,4270,25.88%
2,2022,3,5905,38.29%


In [76]:
client.query('''SELECT year,
                       quarter,
                       FORMAT("%'.2f", total_sales) as total_sales,
                       CONCAT(ROUND(((total_sales 
                                      / LAG(total_sales, 1) OVER (ORDER BY year, quarter)) 
                                      - 1) 
                                      * 100
                                    , 2)
                              , '%') AS month_to_month,
                FROM (SELECT EXTRACT(YEAR FROM created_at) AS year,
                             EXTRACT(QUARTER FROM created_at) AS quarter,
                             SUM(sale_price) AS total_sales,
                      FROM `bigquery-public-data.thelook_ecommerce.order_items`
                      WHERE status='Complete' AND created_at
                       # BETWEEN equals to >= Q1 2022 AND <= Q3 2022
                       BETWEEN "2022-01-01" AND "2022-09-30"
                      # EXTRACT is an agg function, so we need to agg it with GROUP BY
                      GROUP BY year, quarter)
                ORDER BY year ASC, quarter ASC''') \
      .to_dataframe()

,year,quarter,total_sales,month_to_month
0,2022,1,"301,134.16",None
1,2022,2,"382,301.28",26.95%
2,2022,3,"531,421.07",39.01%


In [99]:
# The Look never sold products below the retail price or below the cost / losing money per product.
client.query('''SELECT a.order_id, a.product_id, a.sale_price, b.cost, b.retail_price
                FROM `bigquery-public-data.thelook_ecommerce.order_items` AS a
                INNER JOIN `bigquery-public-data.thelook_ecommerce.products` AS b
                 ON a.product_id=b.id
                WHERE a.sale_price - b.cost < 0
                 OR a.sale_price < b.retail_price
                LIMIT 10''') \
      .to_dataframe()

,order_id,product_id,sale_price,cost,retail_price
